In [52]:
# Function call

import os
import datetime as dt
import pandas as pd
import numpy as np

from ochre import Dwelling
from ochre.utils import default_input_path  # for using sample files
from ochre import HeatPumpWaterHeater



# Define equipment and simulation parameters
setpoint_default = 60  # in C #alternate b/w 60 and 49
deadband_default = 5.56  # in C
max_setpoint = 60
min_setpoint = 49


start_date = dt.datetime(2013, 1, 17, 0, 1) #10441


def predict_two_node(temp_n1, temp_n2, setpoint, draw):
    setpoint_default = setpoint
    print("Simulating Setpoint: ", setpoint_default)
    equipment_args = {
        "start_time": start_date,  # year, month, day, hour, minute
        "time_res": dt.timedelta(minutes=1),
        "duration": dt.timedelta(minutes = 60 * 2 + 15),
        "verbosity": 9,  # required to get setpoint and deadband in results
        "save_results": False,  # if True, must specify output_path
        # "output_path": os.getcwd(),        # Equipment parameters
        "Setpoint Temperature (C)": setpoint_default,
        "Tank Volume (L)": 250,
        "Tank Height (m)": 1.22,
        "UA (W/K)": 2.17,
        "HPWH COP (-)": 4.5,
        "water_nodes": 2
    }

    # Create water draw schedule
    times = pd.date_range(
        equipment_args["start_time"],
        equipment_args["start_time"] + equipment_args["duration"],
        freq=equipment_args["time_res"],
        inclusive="left",
    )
    #withdraw_rate = np.random.choice([0, water_draw_magnitude], p=[0.99, 0.01], size=len(times))
    withdraw_rate = draw
    withdraw_rate = withdraw_rate[:len(times)]
    schedule = pd.DataFrame(
        {
            "Water Heating (L/min)": withdraw_rate,
            "Water Heating Setpoint (C)": setpoint_default,  # Setting so that it can reset
            "Water Heating Deadband (C)": deadband_default,  # Setting so that it can reset
            "Zone Temperature (C)": 20,
            "Zone Wet Bulb Temperature (C)": 15,  # Required for HPWH
            "Mains Temperature (C)": 7,
        },
        index=times,
    )

    # Initialize equipment
    hpwh = HeatPumpWaterHeater(schedule=schedule, **equipment_args)

    hpwh.model.states[:] = np.array([temp_n1, temp_n2])

    # Simulate
    data = pd.DataFrame()
    data = {'draw_data' :[], 'setpoint' :[]}
    control_signal = {}
    setpoints = []

    for t in hpwh.sim_times:
        # Change setpoint based on hour of day
        setpoint = setpoint_default
        control_signal = {
            "Setpoint": setpoint
        }

        setpoints.append(setpoint)
        # Run with controls
        _ = hpwh.update(control_signal=control_signal)

    
    df = hpwh.finalize()

    cols_to_save = [
        "Hot Water Outlet Temperature (C)",
        "T_WH1",
        "T_WH2"
    ]

    to_save = df.loc[:, cols_to_save]
    to_save = to_save[14::15] #Get every 15 minutes
    to_save = to_save[:-1]

    return to_save

In [53]:
#import data
import pandas as pd
import numpy as np

#Data transformation, Y-values are y_i+1 temperatures for the next hour
def rolling_subsets(input_list, window_size = 4):
    if window_size > len(input_list) or window_size <= 0:
        raise ValueError("Window size must be positive and not greater than the list length.")
    
    return [input_list[i:i + window_size] for i in range(int(len(input_list)))]

import numpy as np

def csv_to_features(df, type = 1): #exports features, y = 15 minute increment predictions for 1 hour of outlet temperature
        
    y_col = df["Hot Water Outlet Temperature (C)"]
    window = 10 #2 hours
    result = rolling_subsets(y_col, window)

    power = df["Average Electric Power"]
   

    # Create feature vectors
    x2 = rolling_subsets(df["Draw Data"], window)
    x3 = df["Setpoint"]
    x4 = df["T_WH3"]
    x5 = df["T_WH10"]


    print(len(result), len(x2), len(x3))


    y = result[:-window]  # Remove the first index to offset, y = i+1th prediction

    x2_v = x2[:-window] #draw values
    x3_v = x3[:-window] #setpoint values
    x4_v = x4[:-window] #note temperatures should only be taken at current time interval (live readings)
    x5_v = x5[:-window]

    X = np.column_stack([x2_v, x3_v, x4_v, x5_v]) #Draw value, setpoint, upper node temp, lowe rnode temp
    return X, y


In [54]:
site = 90023
data = pd.read_csv(f"output_site_{site}.csv")
X, y = csv_to_features(data, type)
draws = pd.read_csv(f'..\\ochre\\defaults\\Input Files\\net_flow_{site}.csv')
draws_r = draws.values.transpose().flatten()
draws_r

929236 929236 929236


array([0., 0., 0., ..., 0., 0., 0.])

In [6]:
draws_r = draws.values.transpose().flatten()
draws_r

array([0., 0., 0., ..., 0., 0., 0.])

In [55]:
data #Contains every 15 minute temperatures


,Hot Water Outlet Temperature (C),T_WH3,T_WH10,T_WH12,T_AMB,Water Heating Mode,Average Electric Power,Draw Data,Setpoint
0,48.352419,48.403475,48.403476,48.403475,20.0,Off,0.001,3.78541,49.0
1,48.264842,48.359637,48.359647,48.359254,20.0,Off,0.001,0.00000,49.0
2,48.257316,48.314825,48.315886,48.314165,20.0,Off,0.001,0.00000,49.0
3,48.158983,48.269973,48.272192,48.268165,20.0,Off,0.001,0.00000,49.0
4,48.061725,48.225234,48.228566,48.221314,20.0,Off,0.001,0.00000,49.0
...,...,...,...,...,...,...,...,...,...
929231,60.170479,60.240834,60.213728,55.024011,20.0,Off,0.001,0.00000,59.5
929232,60.031058,60.177599,60.152635,54.922275,20.0,Off,0.001,0.00000,59.5
929233,60.044537,60.080688,60.090944,54.817818,20.0,Off,0.001,0.00000,59.5
929234,59.994424,60.018677,60.028611,54.710885,20.0,Off,0.001,0.00000,59.5


In [57]:
print("MSE 1 Hour: ", mse_1, " RMSE: ", np.sqrt(mse_1) )
print("MSE 2 Hour: ", mse_2, " RMSE: ", np.sqrt(mse_2) )
print("MSE All: ", mse_all, " RMSE: ", np.sqrt(mse_all) )

MSE 1 Hour:  1.1546314615742097  RMSE:  1.07453778973762
MSE 2 Hour:  2.059253538090294  RMSE:  1.4350099435510173
MSE All:  1.2204918691288071  RMSE:  1.104758737973503


In [56]:
from sklearn.metrics import mean_squared_error

mse_1 = 0
mse_2 = 0
mse_all = 0
length = 1000#len(draws_r) - 150
for i in range(length):
    row = X[i]
    draw = draws_r[i:i + 150] #minute level series data
    setpoint = row[-3]
    temp_n1 = row[-2]
    temp_n2 = row[-1]
    #print(i, temp_n1, temp_n2)
    result = predict_two_node(temp_n1, temp_n2, setpoint, draw)

    y_true = data["Hot Water Outlet Temperature (C)"][i + 1: i + 8 + 1].values #Returns temperatures from i + 1st timestep onward
    y_pred = result["Hot Water Outlet Temperature (C)"]
    mse_1 += mean_squared_error([y_true[3]], [y_pred[3]]) # first hour
    mse_2 += mean_squared_error([y_true[7]], [y_pred[7]]) # second hour
    mse_all += mean_squared_error(y_true, y_pred)

mse_1 = mse_1 / length
mse_2 = mse_2 / length
mse_all = mse_all / length


print("MSE 1 Hour: ", mse_1, " RMSE: ", np.sqrt(mse_1) )
print("MSE 2 Hour: ", mse_2, " RMSE: ", np.sqrt(mse_2) )
print("MSE All: ", mse_all, " RMSE: ", np.sqrt(mse_all) )

Simulating Setpoint:  48.99999999999999
2025-06-17 13:50:58.366576 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:50:58.423956 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:50:58.442499 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:50:58.481659 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:50:58.496423 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:50:58.532598 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999


C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_1 += mean_squared_error([y_true[3]], [y_pred[3]]) # first hour
C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_2 += mean_squared_error([y_true[7]], [y_pred[7]]) # second hour
C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a 

2025-06-17 13:50:58.564724 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:50:58.616260 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:50:58.637668 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:50:58.680144 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:50:58.699439 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:50:58.752374 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:50:58.769302 - Heat Pump Water Heater 

C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_1 += mean_squared_error([y_true[3]], [y_pred[3]]) # first hour
C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_2 += mean_squared_error([y_true[7]], [y_pred[7]]) # second hour
C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a 

2025-06-17 13:50:58.820780 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:50:58.837964 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:50:58.876359 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:50:58.893483 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:50:58.936269 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:50:58.951026 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:50:58.991554 - Heat Pump Water Heater 

...

C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_1 += mean_squared_error([y_true[3]], [y_pred[3]]) # first hour
C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_2 += mean_squared_error([y_true[7]], [y_pred[7]]) # second hour
C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a 

2025-06-17 13:52:02.989595 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:52:03.011884 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:52:03.063635 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:52:03.079897 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:52:03.121182 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:52:03.133980 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:52:03.182739 - Heat Pump Water Heater 

C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_1 += mean_squared_error([y_true[3]], [y_pred[3]]) # first hour
C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_2 += mean_squared_error([y_true[7]], [y_pred[7]]) # second hour
C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a 

Simulating Setpoint:  48.99999999999999
2025-06-17 13:52:03.196737 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:52:03.290604 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:52:03.311991 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:52:03.360644 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
Simulating Setpoint:  48.99999999999999
2025-06-17 13:52:03.374936 - Heat Pump Water Heater at 2013-01-17 00:01:00: Initializing Heat Pump Water Heater (OCHRE v0.8.5)
2025-06-17 13:52:03.426523 - Heat Pump Water Heater at 2013-01-17 02:16:00: Simulation complete, time series results saved in memory (not to a file)
MSE 1 Hour:  1.1546314615742097  RMSE:  1.0745377897

C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_1 += mean_squared_error([y_true[3]], [y_pred[3]]) # first hour
C:\Users\janel\AppData\Local\Temp\ipykernel_23508\4212822605.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mse_2 += mean_squared_error([y_true[7]], [y_pred[7]]) # second hour


In [46]:
y_pred
row
i
y[14]
y_pred
y_true
y_true

13    47.693740
14    47.597721
15    47.659237
16    47.563394
17    47.558063
18    47.461596
19    47.366195
20    47.271835
Name: Hot Water Outlet Temperature (C), dtype: float64

In [26]:
data[0:9]

,Hot Water Outlet Temperature (C),T_WH3,T_WH10,T_WH12,T_AMB,Water Heating Mode,Average Electric Power,Draw Data,Setpoint
0,48.352419,48.403475,48.403476,48.403475,20.0,Off,0.001,3.78541,49.0
1,48.264842,48.359637,48.359647,48.359254,20.0,Off,0.001,0.00000,49.0
2,48.257316,48.314825,48.315886,48.314165,20.0,Off,0.001,0.00000,49.0
3,48.158983,48.269973,48.272192,48.268165,20.0,Off,0.001,0.00000,49.0
4,48.061725,48.225234,48.228566,48.221314,20.0,Off,0.001,0.00000,49.0
5,48.126652,48.155661,48.155315,48.173671,20.0,Off,0.001,0.00000,49.0
6,48.029570,48.112209,48.111251,48.125282,20.0,Off,0.001,0.00000,49.0
7,48.024207,48.068087,48.067293,48.076198,20.0,Off,0.001,0.00000,49.0
8,47.926482,48.023592,48.023438,48.026472,20.0,Off,0.001,0.00000,49.0


In [32]:
y_true

0    48.352419
1    48.264842
2    48.257316
3    48.158983
4    48.061725
5    48.126652
6    48.029570
7    48.024207
Name: Hot Water Outlet Temperature (C), dtype: float64

6.666328002632688